# Здесь собрана небольшая аналитика метрик по приложению доставки еды

In [1]:
import pandas as pd
import datetime

In [2]:
path = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-tirel/KC_case_data .csv'

In [3]:
df = pd.read_csv(path, parse_dates=['date'])

In [4]:
df

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads
...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads


In [5]:
# MAU февраля

df \
    .loc[df.date.dt.month == 2] \
    .groupby(['device_id', 'event'], as_index=False) \
    .agg({'os_name': 'nunique'}) \
    .query('event == "app_start"') \
    .shape[0]

75032

In [6]:
df \
    .loc[df.date.dt.month == 1] \
    .groupby(['device_id', 'event'], as_index=False) \
    .agg({'os_name': 'nunique'}) \
    .query('event == "app_install"') \
    .shape[0]

80297

In [7]:
df \
    .groupby(['date', 'device_id', 'event'], as_index=False) \
    .agg({'os_name': 'nunique'}) \
    .query('event == "app_install"') \
    .groupby('date', as_index=False) \
    .agg({'device_id': 'nunique'})

,date,device_id
0,2020-01-01,3579
1,2020-01-02,3144
2,2020-01-03,2402
3,2020-01-04,1831
4,2020-01-05,1671
...,...,...
86,2020-03-27,1199
87,2020-03-28,1091
88,2020-03-29,1117
89,2020-03-30,994


In [8]:
# формирую когорту 

instals = df \
    .groupby(['date', 'device_id', 'event'], as_index=False) \
    .agg({'os_name': 'nunique'}) \
    .query('event == "app_install"') \
    .rename(columns={'date': 'cohort'})
instals.head()

,cohort,device_id,event,os_name
0,2020-01-01,4378,app_install,1
7,2020-01-01,4505,app_install,1
14,2020-01-01,4520,app_install,1
18,2020-01-01,5248,app_install,1
22,2020-01-01,5524,app_install,1


In [9]:
# Первая покупка

first_purchase = df \
    .groupby(['date', 'device_id', 'event'], as_index=False) \
    .agg({'os_name': 'nunique'}) \
    .query('event == "purchase"') \
    .groupby('device_id', as_index=False) \
    .agg({'date': 'min'}) \
    .rename(columns={'date': 'first_purchase'})

In [10]:
df = df.merge(instals[['device_id', 'cohort']], how='left', on='device_id')

In [11]:
df = df.merge(first_purchase[['device_id', 'first_purchase']], how='left', on='device_id')

In [12]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,cohort,first_purchase
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,NaT,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01,NaT
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01,NaT


In [13]:
conversion_quantity = df.loc[((df.first_purchase - df.cohort) >= '0 days') & ((df.first_purchase - df.cohort) <= '7 days')] \
    .groupby('cohort', as_index=False) \
    .agg({'device_id': 'nunique'}) \
    .rename(columns={'device_id': 'seven_days'}) \
    .sort_values('seven_days', ascending=False)
conversion_quantity

,cohort,seven_days
9,2020-01-10,4161
13,2020-01-14,1973
12,2020-01-13,1715
14,2020-01-15,1650
10,2020-01-11,1521
...,...,...
74,2020-03-15,125
73,2020-03-14,120
64,2020-03-05,115
76,2020-03-17,108


In [14]:
conversion = instals \
    .groupby('cohort', as_index=False) \
    .agg({'device_id': 'nunique'}) \
    .rename(columns={'device_id': 'instals'}) \
    .merge(conversion_quantity[['cohort', 'seven_days']], on='cohort', how='left')
conversion.head(15)

,cohort,instals,seven_days
0,2020-01-01,3579,1408
1,2020-01-02,3144,1186
2,2020-01-03,2402,834
3,2020-01-04,1831,639
4,2020-01-05,1671,587
5,2020-01-06,1488,487
6,2020-01-07,1298,423
7,2020-01-08,1154,386
8,2020-01-09,1424,558
9,2020-01-10,13106,4161


In [15]:
conversion = conversion.assign(conversion = (conversion.seven_days / conversion.instals)*100).round(2).sort_values('conversion', ascending=False)

In [16]:
conversion

,cohort,instals,seven_days,conversion
0,2020-01-01,3579,1408,39.34
8,2020-01-09,1424,558,39.19
14,2020-01-15,4310,1650,38.28
13,2020-01-14,5173,1973,38.14
1,2020-01-02,3144,1186,37.72
...,...,...,...,...
77,2020-03-18,1171,235,20.07
88,2020-03-29,1117,223,19.96
81,2020-03-22,1261,251,19.90
87,2020-03-28,1091,209,19.16


# У когорты пользователей, которые установили приложение 2020-01-01 - наибольший показатель конверсии в покупку в первые 7 дней после установки.

In [17]:
df.query('event == "app_install"') \
    .groupby(['device_id', 'utm_source'], as_index=False) \
    .agg({'city': 'count'}) \
    .groupby('utm_source', as_index=False) \
    .agg({'city': 'sum'}) \
    .sort_values('city', ascending=False)

,utm_source,city
0,-,32460
6,yandex-direct,29368
2,google_ads,26286
5,vk_ads,23189
3,instagram_ads,20096
1,facebook_ads,13916
4,referal,9282


In [18]:
register_dates = df.query('event =="register"').drop(columns={'purchase_sum', 'os_name', 'gender', 'city', 'utm_source', 'cohort', 'first_purchase'}).rename(columns={'date': 'registration'})

In [19]:
df = df.merge(register_dates[['device_id', 'registration']], how='left', on='device_id')

In [20]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,cohort,first_purchase,registration
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,NaT,2020-01-01,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01,NaT,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01,NaT,NaT


In [21]:
# отбираю уже зареганных пользователей (по критерию, что нет пустых строк в колонке registration) и по критерию всех действий 
# позже даты регистрации

CR = df.loc[(~df.registration.isna())&(df.registration < df.date)] \
    .groupby('event', as_index=False) \
    .agg({'device_id': 'nunique'}) \
    .sort_values('device_id', ascending=False) \
    .reset_index() \
    .drop(columns='index')

In [22]:
CR

,event,device_id
0,app_start,40991
1,search,40482
2,choose_item,37926
3,tap_basket,34517
4,purchase,24880


In [23]:
(CR.device_id[1] / CR.device_id[0])*100

98.75826400917275

In [24]:
(CR.device_id[2] / CR.device_id[1])*100

93.68608270342375

In [25]:
(CR.device_id[3] / CR.device_id[2])*100

91.01144333702473

In [26]:
(CR.device_id[4] / CR.device_id[3])*100

72.08042413883014

# Больше всего зареганых пользователей отваливается на этапе перехода в корзину

In [27]:
first_purchase_users = df.loc[~df.first_purchase.isna()] \
    .groupby('utm_source', as_index=False) \
    .agg({'device_id': 'nunique'}) \
    .rename(columns={'device_id': 'purchased'})
first_purchase_users

,utm_source,purchased
0,-,21452
1,facebook_ads,11470
2,google_ads,15303
3,instagram_ads,13635
4,referal,7818
5,vk_ads,15565
6,yandex-direct,16259


In [28]:
first_start_users = df \
    .query('event =="app_start"') \
    .groupby('utm_source', as_index=False) \
    .agg({'device_id': 'nunique'}) \
    .rename(columns={'device_id': 'started'})
first_start_users

,utm_source,started
0,-,52268
1,facebook_ads,25957
2,google_ads,38092
3,instagram_ads,31045
4,referal,15924
5,vk_ads,34079
6,yandex-direct,40707


In [29]:
first_start_users = first_start_users.merge(first_purchase_users[['utm_source', 'purchased']], on='utm_source', how='left')

In [30]:
# Пользователи, показавшие наименьшую конверсию в покупку (канал прихода)

first_start_users.assign(CR = (first_start_users.purchased/first_start_users.started)*100).round(2).sort_values('CR', ascending=False)

,utm_source,started,purchased,CR
4,referal,15924,7818,49.10
5,vk_ads,34079,15565,45.67
1,facebook_ads,25957,11470,44.19
3,instagram_ads,31045,13635,43.92
0,-,52268,21452,41.04
2,google_ads,38092,15303,40.17
6,yandex-direct,40707,16259,39.94


# Наименьшую конверсию в первую покупку показали пользователи, пришедшие с Яндекс рекламы

In [31]:
df \
    .query('first_purchase == date and event == "purchase"') \
    .groupby('utm_source', as_index=False) \
    .agg({'purchase_sum': 'median'}) \
    .sort_values('purchase_sum', ascending=False)

,utm_source,purchase_sum
0,-,398.5
4,referal,395.5
3,instagram_ads,393.5
5,vk_ads,393.0
6,yandex-direct,392.5
2,google_ads,390.5
1,facebook_ads,389.0


# Пользователи, пришедшие по реферальной программе, имеют наивысший медианный чек

In [34]:
marketing_data = {'utm_source': ['vk_ads', 'instagram_ads','yandex-direct', 'google_ads', 'facebook_ads'], 
                  'marketing_costs': [9553531, 8561626, 10491707, 10534878, 8590498]}

In [38]:
marketing = pd.DataFrame(marketing_data)
marketing

,utm_source,marketing_costs
0,vk_ads,9553531
1,instagram_ads,8561626
2,yandex-direct,10491707
3,google_ads,10534878
4,facebook_ads,8590498


In [46]:
ROMI = df \
    .groupby('utm_source', as_index=False) \
    .agg({'purchase_sum': 'sum'}) \
    .sort_values('purchase_sum', ascending=False) \
    .merge(marketing[['utm_source', 'marketing_costs']], on='utm_source', how='left')
ROMI.assign(ROMI = (ROMI.purchase_sum-ROMI.marketing_costs)/ROMI.marketing_costs).round(2).sort_values('ROMI', ascending=False)

,utm_source,purchase_sum,marketing_costs,ROMI
1,vk_ads,16389652.5,9553531.0,0.72
2,instagram_ads,14546969.0,8561626.0,0.70
5,facebook_ads,12249901.0,8590498.0,0.43
3,yandex-direct,13915368.0,10491707.0,0.33
4,google_ads,12868276.0,10534878.0,0.22
0,-,21449749.5,NaN,NaN
6,referal,8837044.5,NaN,NaN


# Наибольший показатель ROMI у источника ВК